In [ ]:
from init_notebook import *

In [ ]:
from experiments.datasets import PixelartDataset

ds = PixelartDataset(with_meta=True)
ds[0][1]

In [ ]:
tiles = []
for tile, meta in ds.offset(461):
    if meta["opaque_ratio"] > .9:
        tiles.append(tile[None, ...])
        if len(tiles) >= 58:
            break
tiles = torch.concat(tiles)
VF.to_pil_image(make_grid(tiles, nrow=32))

In [ ]:
LEFT, TOP, BOTTOM, RIGHT = range(4)
match_map = {}
try:
    tiles_mean= tiles.mean()
    for i, tile1 in enumerate(tqdm(tiles)):
        tile1 = tile1 - tiles_mean
        slices1 = [
            #tile1[..., :, 0], tile1[..., 0, :], tile1[..., -1, :], tile1[..., :, -1]
            tile1[..., :, :2], tile1[..., :2, :], tile1[..., -2:, :], tile1[..., :, -2:]
        ]
        for j in range(i, len(tiles)):
            tile2 = tiles[j] - tiles_mean
            slices2 = [
                #tile2[..., :, -1], tile2[..., -1, :], tile2[..., 0, :], tile2[..., :, 0]
                tile2[..., :, -2:], tile2[..., -2:, :], tile2[..., :2, :], tile2[..., :, :2]
            ]
    
            for k, (slice1, slice2) in enumerate(zip(slices1, slices2)):
                diff = (slice1 - slice2).pow(2).mean().sqrt()
                diff = torch.corrcoef(slice1 * slice2)
                if diff > .9:
                    match_map.setdefault(i, {}).setdefault(j, set()).add(k)
                    match_map.setdefault(j, {}).setdefault(i, set()).add(3 - k)
                    
except KeyboardInterrupt:
    pass
num_matches = sum(sum(len(s) for s in v.values()) for v in match_map.values())
print(len(match_map))
print(f"matches {num_matches}, {num_matches / (len(tiles) ** 2 * 4)}")
match_map    

In [ ]:
def render_map(size=(32, 32)):
    map = [[-1] * size[0] for _ in range(size[1])]
    candidates = list(match_map)
    for y in range(size[1]):
        for x in range(size[0]):
            random.shuffle(candidates)
            for idx1 in candidates:
                if x > 0:
                    idx2 = map[y][x - 1]
                    if idx2 >= 0 and LEFT not in match_map[idx1].get(idx2, []):
                        continue
                if y > 0:
                    idx2 = map[y - 1][x]
                    if idx2 >= 0 and TOP not in match_map[idx1].get(idx2, []):
                        continue

                map[y][x] = idx1
                break

    map_tiles = []
    for row in map:
        for idx in row:
            if idx < 0:
                map_tiles.append(torch.zeros(3, 32, 32))
            else:
                map_tiles.append(tiles[idx])
    return VF.to_pil_image(make_grid(map_tiles, nrow=size[0], padding=0))

render_map()